In [12]:
# coding: utf-8

In [13]:
user_config = {
    "account": "pengym_111@163.com",
    "password": "hvwoTxJndBEi8B4G",
    "imap_server": "imap.163.com",
    "imap_port": 993,
    "smtp_server": "smtp.163.com",
    "smtp_port": 25
}

## 接收邮件

In [14]:
import imapy

### 登录

In [15]:
mailbox = imapy.connect(
    host = user_config['imap_server'],
    username = user_config['account'],
    password = user_config['password'],
    ssl = True
)

### 解析邮件内容

In [16]:
def show_messages(messages):
    for msg in messages:
        print('-'*100)
        print('Subject: ', msg['subject'])
        print('From: ', msg['from_email'])
        print('To: ', msg['to'])
        print('Date: ', msg['date'])
        print('*'*100)
        print('Content: ')
        content = msg['text'][0]['text']
        print(content)
        print('Attachments: ', msg['attachments'])
        print('*'*100)

In [17]:
mailbox.folders()

['INBOX', '草稿箱', '已发送', '已删除', '垃圾邮件', '病毒文件夹', '订阅邮件']

In [18]:
messages = mailbox.folder('INBOX').emails()

In [19]:
show_messages(messages)

----------------------------------------------------------------------------------------------------
Subject:  网易邮箱提醒：一次被阻止的收信行为
From:  mail@service.netease.com
To:  "pengym_111@163.com" <pengym_111@163.com
Date:  Mon, 11 Dec 2017 23:38:59 +0800 (CST)
****************************************************************************************************
Content: 
<meta charset="UTF-8" />
<table align="center" style="font-family:Microsoft YaHei,Simsun;width:540px;table-layout:fixed;" bgcolor="#ffffff" cellpadding="0" cellspacing="0">
<tbody>
    <tr>
        <td style="display:none;"></td>
</tr>
<tr>
    <td>
        <table style="width:540px;" border="0" cellpadding="0" cellspacing="0">
<tr>
    <td style="width:540px;height:100px;font-family:Microsoft YaHei,Simsun;background:#ffffff;"><img src="http://mimg.127.net/hxm/dashi/external/notify/14851528754240068.png" style="display:block;border:0;width: 540px;height: 100px;"/></td>
</tr>
</table>
</td>
</tr>
<tr>
    <td>
        <table style

### 根据给定条件过滤邮件

In [20]:
# Get emails in Sent folder:
messages = mailbox.folder('已发送').emails()

In [21]:
show_messages(messages)

----------------------------------------------------------------------------------------------------
Subject:  辣鸡，Project 做完了吗?
From:  pengym_111@163.com
To:  1048217874@qq.com
Date:  Tue, 12 Dec 2017 21:00:21 +0800
****************************************************************************************************
Content: 
hhhhh
Attachments:  [{'filename': 'lenna.jpeg', 'data': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x13\x12\x15\x13\x13\x13\x15\x16\x15\x17\x18\x18\x17\x17\x17\x18\x18\x17\x17\x17\x17\x17\x18\x18\x18\x18\x17\x15\x18\x17\x18\x1d( \x18\x1a%\x1d\x17\x16!1!%)+...\x17\x1f383-7(-.+\x01\n\n\n\x0e\r\x0e\x1b\x10\x10\x1a-%\x1f%//----------/-------------------------------------\xff\xc0\x00\x11\x08\x00\xe1\x00\xe1\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x02\x03\x04\x00\x07\x01\x08\xff\xc4\x00C\x10\x00\x01\x03\x01\x05\x05\x04\x06\

In [22]:
from imapy.query_builder import Q

In [23]:
q = Q()
# Get all unseen emails in inbox
messages = mailbox.folder('INBOX').emails(
    q.unanswered().unseen()
)

In [24]:
show_messages(messages)

## 发送邮件

In [25]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.utils import COMMASPACE, formatdate
import os.path

In [26]:
def send_mail(conf, receivers, subject, content, attachments=None):
    account = conf['account']
    password = conf['password']
    
    message = MIMEMultipart()
    message['From'] = account
    message['To'] = ';'.join(receivers)
    message['Subject'] = subject
    message['Date'] = formatdate(localtime=True)
    message.attach(MIMEText(content))
    
    for f in attachments or []:
        with open(f, 'rb') as fil:
            part = MIMEApplication(
                fil.read(),
                Name = os.path.basename(f)
            )
        part['Content-Disposition'] = 'attachment; filename=%s' % os.path.basename(f)
        message.attach(part)
    
    try:
        smtpObj = smtplib.SMTP()
        smtpObj.connect(user_config['smtp_server'], user_config['smtp_port'])
        smtpObj.login(account, password)
        smtpObj.sendmail(account.format(account), receivers, message.as_string())
        smtpObj.close()
    except smtplib.SMTPException as e:
        print("Error: Unable to send email!")

In [27]:
send_mail(user_config, ['1048217874@qq.com'], '辣鸡，Project 做完了吗?', 'hhhhh', ['lenna.jpeg'])

## UUID生成器

In [28]:
import uuid

In [29]:
code = uuid.uuid1()

In [30]:
print(code)

b0a389ee-e27f-11e7-b002-88b1116f478b
